# Bag of Words 

문서가 가지는 단어의 문맥과 순서를 무시하고 일괄적으로 빈도값을 부여해 피처값을 추출하는 모델

    카운트 기반의 피처 벡터화 : 문서에서 단어가 나타나는 횟수를 부여, 카운트 값이 높을수록 중요하게 인식
    
    TF-IDF 벡터화 : 개별문서에 자주 나타나는 단어에 가중치를 주고, 모든 문서에서 자주 나타나는 단어에는 패널티를 부여

장점 : 쉽고 빠르기 때문에 여러분야에서 활용도가 높다

단점 : 문맥 의미 반영부족, 희소 행렬 문제(대부분의 값이 0으로 채워지는 문제)

# count 및 TF-IDF 벡터화 : CountVectorizer, TfidfVectorizer

입력 파라미터 :
    
    max_df : 너무 높은 빈도수를 가지는 단어 피처를 제외하는 파라미터 (소수점으로 나타낼 경우 상위% 피처는 제외)
    
    min_df : 너무 낮은 빈도수를 가지는 단어 피처를 제외하는 파라미터 (소수점으로 나타낼 경우 하위% 피처는 제외)
    
    max_features : 추출하는 피처의 개수를 제한
    
    stop_words : 'english'로 지정할 경우 영어의 스톤오ㅝ드는 추출에서 제외하는 방식
    
    n_gram_range : n-gram의 범위를 설정 (범위 최솟값,범위 최댓값) 형태로 입력
    
    analyzer : 피처 추출을 수행할 단위를 지정 / 디폴트는 word
    
    token_pattern : 토큰화를 수행하는 정규 표현식 패턴을 지정/ 보통 바꾸지 않음
    
    tokenizer : 토큰화를 별도의 커스텀 함수로 이용시 사용

# BOW 벡터화를 위한 희소 행렬

모든 문서의 있는 단어를 추출해 피처로 벡터화하는 방식은 많은 피처칼럼을 만들게 된다

하지만 각 문서가 가지는 단어의 수는 제한적이기 때문에 대부분 0 값의 행렬을 가지게 된다.

따라서 희소행렬이 물리적으로 적은 메모리 공간을 차지할 수 있도록 COO와 CSR방식의 변환을 이용한다.

**COO 형식 : 0이 아닌 데이터와 그 데이터의 행과 열의 위치를 별도의 데이터 배열에 저장하는 방식**

**CSR 형식 : COO 형식에서 행 위치 배열 내에 있는 고유한 값의 시작 위치를 별도의 위치배열로 가지는 방식**
    
    ex) [0,0,1,1,1,1,1,2,2,3,4,4,5] -> [0,2,7,9,10,12]

In [4]:
# 희소행렬 - COO (scipy의 sparse 이용)
import numpy as np

dense = np.array( [ [ 3, 0, 1 ], [0, 2, 0 ] ] )

from scipy import sparse

# 0 이 아닌 데이터 추출
data = np.array([3,1,2])

# 행 위치와 열 위치를 각각 array로 생성 
row_pos = np.array([0,0,1])
col_pos = np.array([0,2,1])

# sparse 패키지의 coo_matrix를 이용하여 COO 형식으로 희소 행렬 생성
sparse_coo = sparse.coo_matrix((data, (row_pos,col_pos)))
print(sparse_coo)

  (0, 0)	3
  (0, 2)	1
  (1, 1)	2


sparse_coo는 COO형식의 희소 행렬 객체 변수를 의미한다.

In [6]:
# 희소행렬 - CRS (scipy의 csr_matrix 이용)
from scipy import sparse

dense2 = np.array([[0,0,1,0,0,5],
             [1,4,0,3,2,5],
             [0,6,0,3,0,0],
             [2,0,0,0,0,0],
             [0,0,0,7,0,8],
             [1,0,0,0,0,0]])

# 0 이 아닌 데이터 추출
data2 = np.array([1, 5, 1, 4, 3, 2, 5, 6, 3, 2, 7, 8, 1])

# 행 위치와 열 위치를 각각 array로 생성 
row_pos = np.array([0, 0, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 5])
col_pos = np.array([2, 5, 0, 1, 3, 4, 5, 1, 3, 0, 3, 5, 0])

# COO 형식으로 변환 
sparse_coo = sparse.coo_matrix((data2, (row_pos,col_pos)))

# 행 위치 배열의 고유한 값들의 시작 위치 인덱스를 배열로 생성
row_pos_ind = np.array([0, 2, 7, 9, 10, 12, 13])

# CSR 형식으로 변환 
sparse_csr = sparse.csr_matrix((data2, col_pos, row_pos_ind))
print(sparse_csr)

  (0, 2)	1
  (0, 5)	5
  (1, 0)	1
  (1, 1)	4
  (1, 3)	3
  (1, 4)	2
  (1, 5)	5
  (2, 1)	6
  (2, 3)	3
  (3, 0)	2
  (4, 3)	7
  (4, 5)	8
  (5, 0)	1


In [7]:
# 실제로 희소행렬을 사용할 때
dense3 = np.array([[0,0,1,0,0,5],
             [1,4,0,3,2,5],
             [0,6,0,3,0,0],
             [2,0,0,0,0,0],
             [0,0,0,7,0,8],
             [1,0,0,0,0,0]])

coo = sparse.coo_matrix(dense3)
csr = sparse.csr_matrix(dense3)